# Alinhamento múltiplo do gene AT10A

In [68]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
import requests
from io import StringIO                  

Dados da proteina através do Uniprot 

In [69]:
def fetch_protein_sequence(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    response = requests.get(url)
    
    if response.ok:
        # Extract the protein sequence from the FASTA format
        fasta_data = response.text
        fasta_io = StringIO(fasta_data)
        record = SeqIO.read(fasta_io, "fasta")
        return record.seq
    else:
        print(f"Failed to fetch protein sequence from {url}.")
        return None

Obter os resultados do BlastP

In [70]:
def run_blast(sequence, blast_program="blastp", database="swissprot", outfile="blast_results.xml"):
    # Run BLAST with the given sequence
    result_handle = NCBIWWW.qblast(blast_program, database, sequence)
    
    # Save the results to a file
    with open(outfile, "w") as out_file:
        out_file.write(result_handle.read())
    
    result_handle.close()

Salvar os resultados do Blast num ficheiro xml de nome "blast_results"

In [71]:
if __name__ == "__main__":
    uniprot_id = "O60312"  # UniProt ID of the protein you want to fetch and BLAST

    # Fetch protein sequence from UniProt
    protein_sequence = fetch_protein_sequence(uniprot_id)

    if protein_sequence:
        print(f"Protein sequence for {uniprot_id}:\n{protein_sequence}")

        # Run BLAST with the obtained protein sequence
        run_blast(protein_sequence, blast_program="blastp", database="swissprot", outfile="blast_results.xml")

        print(f"BLAST results saved to blast_results.xml")

Protein sequence for O60312:
MEREPAGTEEPGPPGRRRRREGRTRTVRSNLLPPPGAEDPAAGAAKGERRRRRGCAQHLADNRLKTTKYTLLSFLPKNLFEQFHRPANVYFVFIALLNFVPAVNAFQPGLALAPVLFILAITAFRDLWEDYSRHRSDHKINHLGCLVFSREEKKYVNRFWKEIHVGDFVRLRCNEIFPADILLLSSSDPDGLCHIETANLDGETNLKRRQVVRGFSELVSEFNPLTFTSVIECEKPNNDLSRFRGCIIHDNGKKAGLYKENLLLRGCTLRNTDAVVGIVIYAGHETKALLNNSGPRYKRSKLERQMNCDVLWCVLLLVCMSLFSAVGHGLWIWRYQEKKSLFYVPKSDGSSLSPVTAAVYSFLTMIIVLQVLIPISLYVSIEIVKACQVYFINQDMQLYDEETDSQLQCRALNITEDLGQIQYIFSDKTGTLTENKMVFRRCTVSGVEYSHDANAQRLARYQEADSEEEEVVPRGGSVSQRGSIGSHQSVRVVHRTQSTKSHRRTGSRAEAKRASMLSKHTAFSSPMEKDITPDPKLLEKVSECDKSLAVARHQEHLLAHLSPELSDVFDFFIALTICNTVVVTSPDQPRTKVRVRFELKSPVKTIEDFLRRFTPSCLTSGCSSIGSLAANKSSHKLGSSFPSTPSSDGMLLRLEERLGQPTSAIASNGYSSQADNWASELAQEQESERELRYEAESPDEAALVYAARAYNCVLVERLHDQVSVELPHLGRLTFELLHTLGFDSVRKRMSVVIRHPLTDEINVYTKGADSVVMDLLQPCSSVDARGRHQKKIRSKTQNYLNVYAAEGLRTLCIAKRVLSKEEYACWLQSHLEAESSLENSEELLFQSAIRLETNLHLLGATGIEDRLQDGVPETISKLRQAGLQIWVLTGDKQETAVNIAYACKLLDHDEEVITLNATSQEACAALLDQCLCYVQSRGLQRAPEKTKGKVSMRFSSLCPPSTSTASGRRPS

Abrir o ficheiro xml de nome "blast_results" e lê-lo

In [72]:
result_handle=open("blast_results.xml")
record=NCBIXML.read(result_handle)

Parâmetros globais da pesquisa realizada no Blast

In [73]:
print("A base de dados utilizada foi", record.database)
print("A matriz de substituição utilizada no alinhamento foi", record.matrix)
print("Os parâmetros de espaçamento do alinhamento são", record.gap_penalties)

A base de dados utilizada foi swissprot
A matriz de substituição utilizada no alinhamento foi BLOSUM62
Os parâmetros de espaçamento do alinhamento são (11, 1)


Número de sequências homólogas da sequência nucleotídica em questão encontradas pelo BLAST, ou seja, o número de alinhamentos retornados (número de hits).

In [74]:
nhits=len(record.alignments)
print("O número de alinhamentos retornados é", nhits)

O número de alinhamentos retornados é 50


Identificar a lista de acession numbers, os e-values, os comprimentos dos alinhamentos e a identificação das sequências homólogas dos alinhamentos obtidas na pesquisa

In [80]:
resultado=[]
for alignment in record.alignments:
    evalue=alignment.hsps[0].expect
    score = alignment.hsps[0].score
    identities = alignment.hsps[0].identities
    accession=alignment.accession
    length=alignment.hsps[0].align_length
    sequence=alignment.title
    resultado.append("\n" + "\n" + str(accession) + " > " + " e-value:"+ str(evalue) + "," + " score: " + str(score) + "," + " identities: " + str(identities) + " e" + " comprimento:" + str(length) + "\n" + "\nsequência:" + str(sequence) )

print("Os e-values e comprimento dos alinhamentos obtidos para cada sequência são")
print(" ")
for s in resultado:
    print(s)

Os e-values e comprimento dos alinhamentos obtidos para cada sequência são
 


O60312 >  e-value:0.0, score: 8049.0, identities: 1499 e comprimento:1499

sequência:sp|O60312.2| RecName: Full=Phospholipid-transporting ATPase VA; AltName: Full=ATPase class V type 10A; AltName: Full=Aminophospholipid translocase VA; AltName: Full=P4-ATPase flippase complex alpha subunit ATP10A [Homo sapiens]


O54827 >  e-value:0.0, score: 6355.0, identities: 1203 e comprimento:1477

sequência:sp|O54827.4| RecName: Full=Phospholipid-transporting ATPase VA; AltName: Full=ATPase class V type 10A; AltName: Full=P-locus fat-associated ATPase; AltName: Full=P4-ATPase flippase complex alpha subunit ATP10A [Mus musculus]


Q9P241 >  e-value:0.0, score: 3575.0, identities: 704 e comprimento:1316

sequência:sp|Q9P241.3| RecName: Full=Phospholipid-transporting ATPase VD; AltName: Full=ATPase class V type 10D; AltName: Full=P4-ATPase flippase complex alpha subunit ATP10D [Homo sapiens]


Q8K2X1 >  e-value:0.0, score